## 1. Pythonの準備とGAアクセス

PythonのseleniumとChromeDriverを事前にインストールしておく。
- pip install selenium
- pip install chromedriver_binary

動作確認したのはMacのChromeのみ。

In [ ]:
dir_chrome_prof = '/Users/mak/Dropbox/Apps/Chrome-User-Data/mak00s'
dir_download    = '/Users/mak/Downloads/auto'
file_csv = dir_download + '/user-report-ALL-processed.csv'

url_ga = 'https://analytics.google.com/analytics/web/'

from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# Chromeの準備
options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=' + dir_chrome_prof)
prefs = {"download.default_directory": dir_download}
options.add_experimental_option('prefs', prefs)
browser = webdriver.Chrome(options = options)
#browser.implicitly_wait(10)

# Google Analyticsにアクセス
browser.get(url_ga)

## 2. GAにログイン
別ウィンドウでChromeが起動するので、手動でログインし、該当のビューに切り替えてから次へ進む。

In [ ]:
username = 'xxx@gmail.com'
#keyring.set_password('google', username, 'xxx') # 最初のみパスワードを保存しておく

# ログイン
import keyring
pwd = keyring.get_password('google', username) # 2回目以降はパスワードを取り出せる

inputElement = browser.find_element_by_id("identifierId")
inputElement.send_keys(username)
inputElement.send_keys(Keys.ENTER)

## 3. GAを自動操作してJSONをDownload
ユーザーエクスプローラの画面を開く。

In [ ]:
# URLからIDを取得
import re
m = re.search('/([a-z0-9]+)/?$', browser.current_url)
if m:
    reportId = m.group(1)
    print(reportId)
    # User Explorerの画面を開く
    browser.get(url_ga + '#/report/visitors-legacy-user-id/' + reportId + '/')

ユーザーエクスプローラの画面が開いたら、日付やセグメント、「表示する行数」を必要に応じて変更しておく。この画面に表示されたClient IDを取得することになる。

注意：Chromeの場合、重複ファイル名の自動変更は99までであり、101ファイル目で保存ダイアログが出るがクリックは継続してしまうので、行数は100を推奨。

In [ ]:
# Client IDのリストを画面から取得
iframe=browser.find_element_by_xpath('//iframe[@id="galaxyIframe"]')
browser.switch_to.frame(iframe)
elem = browser.find_elements_by_css_selector('#ID-rowTable div.C_USER_LIST_TEXT_DIV')
cid = []
for index in range(len(elem)):
    cid.append(browser.find_elements_by_css_selector('#ID-rowTable div.C_USER_LIST_TEXT_DIV')[index].text)

print(str(len(cid)) + ' Client IDs were identified: ' + cid[0] + ' - ' + cid[-1])

In [ ]:
# リストからIDを見つけてクリックしていく
import sys
downloaded = []
for i in range(len(cid)):
    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, 'ID-rowTable')))
        try:
            WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[text()="' + cid[i] + '"]'))).click()
            WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-name='ID-export']"))).click()
            print('Downloading ' + str(i+1) + '/' + str(len(cid)) + ': ' + cid[i])
            downloaded.append(cid[i])
            WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ACTION-back"))).click()
        except:
            print('Skipping ' + str(i+1) + ' due to error ', sys.exc_info()[0])
    except:
        print("Couldn't get back to a list of Client IDs.", sys.exc_info()[0])
print(str(len(downloaded)) + ' JSON files were downloaded.')

## 4. Downloadされた複数のJSONファイルを加工

JSONをフラットに開く参考：
- https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json/39906235
- https://note.nkmk.me/python-pandas-json-normalize/
- https://www.dataquest.io/blog/python-json-tutorial/

In [ ]:
import pandas as pd
#pd.options.display.html.table_schema = True
from pandas.io.json import json_normalize
import json
import glob
import os.path

# ファイルを読み込む
files = glob.glob(dir_download + '/user-report-export*.json').sort(key=os.path.getmtime)
df2 = pd.DataFrame()

for file_ in files:
    print('opening ' + file_.split('/')[-1])
    with open(file_, 'r') as f:
        data = json.load(f)
    # 最下層をレコードの単位にしつつpandasのDataFrameに変換する
    df = json_normalize(data['dates'], record_path=['sessions', 'activities', 'details'],
                              meta=['date', ['sessions', 'channel'], ['sessions', 'deviceCategory'],
                                   ['sessions', 'activities', 'time']])    
    # カラム名を変換
    d = {'ページの': 'Page', 'タイトル': ' title', 'イベント( |の)?': 'Event ',
         'カテゴリ': 'category', 'アクション': 'action', 'ラベル': 'label',
         '値': 'value', '目標( |の)?': 'Goal ', '完了数': 'completion(s)',
         '番号': 'index', '名': 'name', '数': 'count'}
    df.columns = df.columns.to_series().replace(d, regex=True)
    # ファイル名のカラムを追加
    df['User'] = re.sub(r'^.*\/', '', file_)
    # 値を変換
    df.replace({'sessions.activities.time': r'午前'}, {'sessions.activities.time': 'AM'}, inplace=True, regex=True)
    df.replace({'sessions.activities.time': r'午後'}, {'sessions.activities.time': 'PM'}, inplace=True, regex=True)
    df['Page URL'] = df['Page URL'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['Page title'] = df['Page title'].apply(lambda x: x[0] if isinstance(x, list) else x)
    # 日付と時間を統合
    df['dateTime'] = pd.to_datetime(df['date'] + ' ' + df['sessions.activities.time'])
    df.drop(['date', 'sessions.activities.time'], axis=1, inplace=True)
    # カラム名を変更
    df.rename(columns={'sessions.channel': 'Channel',
                       'sessions.deviceCategory': 'Device'}, inplace=True)
    # データフレームを結合
    df2 = pd.concat([df2, df], sort=False).sort_values(['User','dateTime'])

# ページURLからindex.htmlをカット
df2['Page URL'].replace('index.html', '', regex=True, inplace=True)
# URLパラメータを削除
df2['Page URL'].replace('[\?&](fbclid|paged|page|rk|no)=.*', '', regex=True, inplace=True)
# ページタイトルからサイト名をカット
df2['Page title'].replace(' ?[|｜│-][^|｜│-]+$', '', regex=True, inplace=True)
# CSVファイル名のうち数字部分のみをUserとする
df2['User'].replace('[^\d]', '', regex=True, inplace=True)
# Userごとに連番をふる
df2['order'] = df2.groupby('User').cumcount() + 1
# Eventの配列を結合しフラットに
df2['Event action']=df2['Event action'].apply(lambda x:",".join(x) if pd.notnull(x) else x)
df2['Event category']=df2['Event category'].apply(lambda x:",".join(x) if pd.notnull(x) else x)
df2['Event label']=df2['Event label'].apply(lambda x:",".join(x) if pd.notnull(x) else x)

## 5. CSV保存

In [ ]:
# 必要なカラムだけ残してCSV保存
df2[['User','Channel','Device','order','dateTime','Page URL',
     'Page title','Event category','Event action','Event label']
   ].to_csv(file_csv, header=True, index=False)